In [2]:
import multiprocessing
from PIL import Image
import os.path
import numpy as np
import pandas as pd
import plotly.express as px
import torch
# from numpy import dtype
# import time
from matplotlib import pyplot as plt
from metavision_core.event_io.raw_reader import initiate_device
from metavision_core.event_io.raw_reader import RawReader
%config InlineBackend.figure_formats = ['svg']

# Read the Raw event data and Trig time

The trig time indicating when the AWG starts sweeping


In [4]:
raw_data=pd.read_csv('E:\\phd\\program\\EventCam_220330\\20mW_7ms_forward_thr75.0_fo1725_AWG_2835-2905-test2.csv')
TrigEvent=pd.read_csv('E:\\phd\\program\\EventCam_220330\\20mW_7ms_forward_thr75.0_fo1725_AWG_2835-2905_trig-test2.csv')

# Cut a section of the raw event (saved as raw data2) to do further process

In [5]:
XSIZE=640 # frame size
YSIZE=480
num_loop=1 # numer of looped Microwave (MW) frequency sweeping 
num_repeat=1 # repeating the above mentioned loop sweeping
period=7000 # sweeping period; unit: us 
cycle_start=20 # discard the first 20 sweep loops
total_cycle=num_loop*num_repeat*2 # in TrigEvents, every cycle contains 2 trig evnts. Either can be used
# period_selection=np.arange(cycle_start,total_cycle+cycle_start,2).reshape((-1,1)) #
trig_start = TrigEvent['t'][cycle_start]
trig_end= trig_start+period*num_loop*num_repeat
filter=(raw_data['t']>=trig_start)&(raw_data['t']<=trig_end)
raw_data2=raw_data[filter]
# shift_temp=raw_data2['t']-raw_data2['t'][0]
# shift_temp=raw_data2[0]['t']
# raw_data2['t']=raw_data2['t']-raw_data2['t'][0]
raw_data2['t']=raw_data2['t']-trig_start  # shift the time to 0
raw_values = (raw_data2['p'] - 0.5) * 2  # Convert (0, 1) to (-1, 1)
raw_data2['p'] = raw_values.astype(np.int32)

C:\Users\eee\AppData\Local\Temp\ipykernel_17592\249686785.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data2['t']=raw_data2['t']-trig_start  # shift the time to 0
C:\Users\eee\AppData\Local\Temp\ipykernel_17592\249686785.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data2['p'] = raw_values.astype(np.int32)


In [6]:
print(type(raw_data))
print(type(raw_data2))
print(raw_data2.shape)


raw_data2['t'].to_numpy()[0]

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
(15359, 4)


0

# Frequency sweeping range 

# Do filtering by 3D convolution on raw events data
    the 3D convolution is realized by torch.sparse.sum with 
    x kernel size = 2*x_length, x_stride=1 
    y kernel size = 2*y_length, y_stride=1
    t kernel size= t_length, t_stride = t_length
    

In [7]:

# key parameters to do the 3D convolution:
x_length=5 # x kernel size
y_length=5 # y kernel size
t_length=20 # t kernel size
x_center_set=np.arange(x_length,XSIZE-x_length+1,1) # central pixel address to be convolutioned; 
y_center_set=np.arange(y_length,YSIZE-y_length+1,1)


event_total=[] # array storing all convolution results
num_l=0

# sweep across the frame pixels:
for j, y_center in enumerate(y_center_set):
    for i, x_center in enumerate(x_center_set):
        
        events_temp=np.zeros((period//t_length, 4)) # array to store convolution result mapped to (y_center, x_center); 1st column- 'x', 2nd column - 'y', 3rd column- 't', 4th column - 'p'
        events_temp[:,0]=x_center
        events_temp[:,1]=y_center
        events_temp[:,2]=np.arange(0,period,t_length)
        filter=(raw_data2['x']>=(x_center-x_length))&(raw_data2['x']<(x_center+x_length))&(raw_data2['y']>=(y_center-y_length))&(raw_data2['y']<(y_center+y_length)) # choose local areas of 2*y_length by 2* x_length centered at (y_center, x_center) to do convolution
        if max(filter)==True:  # some areas may contains no events
            # raw_data_temp=raw_data2[filter]
            raw_values = raw_data2[filter]['p'].to_numpy()
            raw_indexes = np.array([
                        raw_data2[filter]['t'].to_numpy(), 
                        raw_data2[filter]['x'].to_numpy()-(x_center-x_length), # shift to 0
                        raw_data2[filter]['y'].to_numpy()-(y_center-y_length), 
                        ])
            raw_data_sparse= torch.sparse_coo_tensor(raw_indexes, raw_values,size=(period*num_loop,2*x_length,2*y_length))
            raw_data_xy_bining=torch.sparse.sum(torch.sparse.sum(raw_data_sparse, 2),dim=1)
            raw_data_t_bining=torch.sum(raw_data_xy_bining.to_dense().reshape((num_loop,period)),dim=0) # stack events of num_loop sweepings 
            raw_data_t_bining=torch.sum(raw_data_t_bining.reshape((period//t_length,t_length)),dim=1) # sum events within every t_length range
            events_temp[:,3]=raw_data_t_bining
            events_temp=events_temp[abs(events_temp[:,3])>1] # keep results of which p>1; can be changed to modify the display effect
            event_total.extend(events_temp)


# Convert results to numpy array and save data


In [8]:
event_total=np.array(event_total)
df = pd.DataFrame(event_total,columns=['x','y','t','p'])
df.to_csv('E:\\phd\\program\\EventCam_220330\\20mW_70ms_forward_thr75.0_fo1725_AWG_2835-2905-Convolution_test.csv',index=False)
df.tail()

,x,y,t,p
80,565.0,405.0,860.0,2.0
81,145.0,415.0,100.0,2.0
82,355.0,425.0,820.0,2.0
83,215.0,455.0,600.0,2.0
84,355.0,465.0,480.0,2.0


# Plot result in 3D

In [9]:

fig = px.scatter_3d(df, x='t', y='x', z='y', color='p', 
                    opacity=0.3,
                    width=1400, height=700
                )
fig.update_traces(marker_size = 0.5)

fig.update_layout(scene_aspectmode='manual',
                  scene_aspectratio=dict(x=3, y=1, z=1))

fig.show() 